#  Catálogo Clientes

## 1. Coluna `nome`
- Remoção de caracteres invisíveis (`\u00A0`, `\u2000-\u200B`).
- Aplicação de `trim` para retirar espaços extras.
- Conversão para formato capitalizado com `initcap`.

---

## 2. Coluna `cidade`
- Remoção de caracteres invisíveis.
- Eliminação de prefixos como **da**, **de**, **do**, **das** no início do texto.
- Aplicação de `trim` e `initcap` para padronização.

---

## 3. Coluna `email`
- Remoção de espaços extras com `trim`.
- Conversão para letras maiúsculas com `upper`.

---

## 4. Tratamento de Valores Nulos
- Substituição de valores `NULL`:
  - `email` → `"Desconhecido"`
  - `nome` → `"Desconhecido"`
  - `cidade` → `"Não Informado"`
- Remoção de linhas sem `id` (`na.drop(subset=["id"])`).

---



In [0]:
# Forma correta para ler uma tabela Delta registrada no catálogo/schema
df1 = spark.table("hive_metastore.bronze.vendas_clientes")
display(df1)


# df.filter(col("nome").isNull()).show();

# #Filtra registros sem valores nulos 
# df.filter(col("nome").isNotNull()).show();

# #Conta quantos valores nulos existem em uma coluna 
# df.filter(col("nome").isNull()).count();

# # Filtra valrias colunas nulas de uma vez 
# df.filter(col("nome").isNull() | col("cidade").isNull()).show();




In [0]:
from pyspark.sql.functions import col, trim, initcap, regexp_replace, upper

df1 = (
    df1.withColumn(
        "nome",
        initcap(
            trim(
                regexp_replace(col("nome"), r"[\u00A0\u2000-\u200B]", " ")
            )
        )
    )
    .withColumn(
        "cidade",
        initcap(
            trim(
                regexp_replace(
                    regexp_replace(col("cidade"), r"[\u00A0\u2000-\u200B]", " "), 
                    r"^(da|de|do|das)\s+", ""
                )
            )
        )
    )
    .withColumn("email", upper(trim(col("email"))))
    .fillna({"email": "Desconhecido", "nome": "Desconhecido", "cidade": "Não Informado"})
    .na.drop(subset=["id"])
    .dropDuplicates(["id"])   
)

display(df1)


In [0]:
# %sql
# CREATE DATABASE IF NOT EXISTS silver;

# CREATE TABLE IF NOT EXISTS silver.vendas_clientes (
#   id INT,
#   nome STRING,
#   email STRING,
#   cidade STRING
# )



In [0]:
df1.write \
    .format("delta") \
    .mode("append") \
    .saveAsTable("silver.vendas_clientes")

# Catálogo Produtos

## 1. Leitura da Tabela Bronze
- Carregamento da tabela **`hive_metastore.bronze.vendas_produtos`** como fonte inicial.

---

## 2. Separação da Coluna `nome`
- Divisão da coluna `nome` em duas partes:
  - **`categoria`**: primeira palavra.
  - **`numero`**: segunda palavra, convertida para inteiro.

---

## 3. Seleção de Colunas
- Mantidas apenas as colunas relevantes: `id`, `nome`, `categoria`, `numero`, `preco`.

---

## 4. Correção de Categorias
- Criada a coluna **`categoria_corrigida`** com valores padronizados.
- Correção de erros de digitação (ex.: "movel" → "Móvel", "eletronico" → "Eletrônico").
- Tratamento de valores nulos: substituídos por "Desconhecido".

---

## 5. Criação de `nome_corrigido`
- Construída a coluna **`nome_corrigido`** concatenando `categoria_corrigida` e `numero`.

---

## 6. Limpeza de Dados
- Removidos registros com `id` nulo.
- Removidos espaços extras em `nome_corrigido` e `categoria_corrigida`.

---

## 7. Escrita na Tabela Silver
- Persistência dos dados transformados na tabela **`silver.vendas_produtos`**.
- Formato: **Delta**.
- Modo: **append** (acréscimo de registros).

---



In [0]:
df2 = spark.table("hive_metastore.bronze.vendas_produtos")
display(df2)


In [0]:
from pyspark.sql.functions import split, col

# separa em duas partes: antes e depois do espaço
df2 = (
    df2.withColumn("categoria", split(col("nome"), " ")[0])
        .withColumn("numero", split(col("nome"), " ")[1].cast("int"))
        .select("id", "nome", "categoria", "numero", "preco")
        .dropDuplicates(["id"])  
)

display(df2)


In [0]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType

correcoes = {
    
    "movel": "Móvel",
    "móvei": "Móvel",  
    "eletronico": "Eletrônico",
    "brinquedo": "Brinquedo",
    "alimento": "Alimento",
    "roupa": "Roupa",
    "livro": "Livro"


}

def corrigir_categoria(cat):
    if cat is None:
        return "Desconhecido"
    return correcoes.get(cat.lower(), cat)

corrigir_udf = udf(corrigir_categoria, StringType())

df2 = df2.withColumn("categoria_corrigida", corrigir_udf(col("categoria")))

df2 = df2.select("id","nome","categoria_corrigida", "numero", "preco")

display(df2)

In [0]:
from pyspark.sql.functions import concat_ws

df2 = df2.withColumn(
    "nome_corrigido",
    concat_ws(" ", col("categoria_corrigida"), col("numero"))
)

df2 = df2.select("id", "nome_corrigido","categoria_corrigida")

display(df2)


In [0]:
from pyspark.sql.functions import when, col,trim, abs , round

df2 = df2.select("id", "nome_corrigido","categoria_corrigida")

df2 = df2.na.drop(subset=["id"])

df2 = df2.withColumn("nome_corrigido", trim(col("nome_corrigido"))) \
         .withColumn("categoria_corrigida", trim(col("categoria_corrigida")))

display(df2)

df2.write \
    .format("delta") \
    .mode("append") \
    .saveAsTable("silver.vendas_produtos")


#Catálogo itens pedidos

## 1. Leitura da Tabela Bronze
- Carregamento da tabela **`hive_metastore.bronze.vendas_itens__pedidos`** como fonte inicial.

---

## 2. Remoção de Registros Inválidos
- Exclusão de linhas com **`id` nulo** para garantir integridade dos dados.

---

## 3. Correção da Coluna `preco_unitario`
- Criada a coluna **`preco_unitario_corrigido`**:
  - Valores nulos substituídos por **0**.
  - Valores negativos transformados em positivos com `abs`.
  - Arredondamento aplicado para **2 casas decimais**.

---

## 4. Correção da Coluna `quantidade`
- Criada a coluna **`quantidade_corrigida`**:
  - Valores nulos substituídos por **0**.
  - Valores válidos mantidos sem alteração.

---

## 5. Seleção de Colunas Relevantes
- Mantidas apenas as colunas necessárias para análise:
  - `id`
  - `pedido_id`
  - `produto_id`
  - `quantidade_corrigida`
  - `preco_unitario_corrigido`

---

## 6. Escrita na Tabela Silver
- Persistência dos dados transformados na tabela **`silver.vendas_itens__pedidos`**.
- Formato: **Delta**.
- Modo: **append** (acréscimo de registros).

---



In [0]:
from pyspark.sql.functions import col, when, abs, round

df3 = spark.table("hive_metastore.bronze.vendas_itens__pedidos")

df3 = df3.na.drop(subset=["id"])

df3 = (
    df3
    .withColumn(
        "preco_unitario_corrigido",
        when(col("preco_unitario").isNull(), 0)
        .otherwise(abs(col("preco_unitario")))
    )
    .withColumn(
        "quantidade_corrigida",
        when(col("quantidade").isNull(), 0)
        .otherwise(col("quantidade"))
    )
)

df3 = df3.withColumn("preco_unitario_corrigido", round(col("preco_unitario_corrigido"), 2))

df3 = (
    df3.select("id", "pedido_id", "produto_id", "quantidade_corrigida", "preco_unitario_corrigido")
        .dropDuplicates(["id"])   
)

display(df3)
df3.write \
    .format("delta") \
    .mode("append") \
    .saveAsTable("silver.vendas_itens__pedidos")


#catálogo pedidos



## 1. Leitura da Tabela Bronze
- Carregamento da tabela **`hive_metastore.bronze.vendas_pedidos`** como fonte inicial.

---

## 2. Seleção de Colunas
- Mantidas apenas as colunas relevantes para análise:
  - `id`
  - `cliente_id`
  - `data`

---

## 3. Remoção de Registros Inválidos
- Exclusão de linhas com **`id` nulo** para garantir integridade dos dados.

---

## 4. Escrita na Tabela Silver
- Persistência dos dados transformados na tabela **`silver.vendas_pedidos`**.
- Modo: **append** (acréscimo de registros).

---



In [0]:
df4 = spark.table("hive_metastore.bronze.vendas_pedidos")

df4 = df4.select("id", "cliente_id", "data")

df4 = df4.na.drop(subset=["id"])

df4 = df4.dropDuplicates(["id"])

display(df4)

df4.write \
    .format("delta") \
    .mode("append") \
    .saveAsTable("silver.vendas_pedidos")
